# Load the TFLearn model

In [ ]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
import tensorflow as tf
import scipy
import numpy as np
import pandas as pd
import os
import glob
from skimage.io import imread
import skimage
import matplotlib.pyplot as plt

import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
DATABASE_PATH = 'C:\\Users\\williamrobbn\\SilCam\\PySilCam\\pysilcam-testdata\\unittest-data\\silcam_classification_database'
MODEL_PATH = 'C:\\Users\\williamrobbn\\SilCam\\PySilCam\\pysilcam-testdata\\tflmodel\\particle-classifier.tfl'

In [ ]:
def get_class_labels(model_path='/mnt/ARRAY/classifier/model/particle-classifier.tfl'):
    path, filename = os.path.split(model_path)
    header = pd.read_csv(os.path.join(path, 'header.tfl.txt'))
    class_labels = header.columns
    return class_labels

def load_model(model_path='/mnt/ARRAY/classifier/model/particle-classifier.tfl'):
    path, filename = os.path.split(model_path)
    header = pd.read_csv(os.path.join(path, 'header.tfl.txt'))
    OUTPUTS = len(header.columns)
    class_labels = header.columns

    tf.reset_default_graph()

    # Same network definition as in tfl_tools scripts
    img_prep = ImagePreprocessing()
    img_prep.add_featurewise_zero_center()
    img_prep.add_featurewise_stdnorm()
    img_aug = ImageAugmentation()
    img_aug.add_random_flip_leftright()
    img_aug.add_random_rotation(max_angle=25.)
    img_aug.add_random_blur(sigma_max=3.)

    input1 = input_data(shape=[None, 32, 32, 3],
                         data_preprocessing=img_prep,
                         data_augmentation=img_aug)
    conv1 = conv_2d(input1, 32, 3, activation='relu')
    pool1 = max_pool_2d(conv1, 2)
    conv2 = conv_2d(pool1, 64, 3, activation='relu')
    conv3 = conv_2d(conv2, 64, 3, activation='relu')
    conv4 = conv_2d(conv3, 64, 3, activation='relu')
    conv5 = conv_2d(conv4, 64, 3, activation='relu')
    conv6 = conv_2d(conv5, 64, 3, activation='relu')
    pool2 = max_pool_2d(conv6, 2)
    fc1 = fully_connected(pool2, 512, activation='relu')
    drop1 = dropout(fc1, 0.75)
    fc2 = fully_connected(drop1, OUTPUTS, activation='softmax')
    network = regression(fc2, optimizer='adam',
                         loss='categorical_crossentropy',
                         learning_rate=0.001)

    model = tflearn.DNN(network, tensorboard_verbose=0,
            checkpoint_path=model_path)
    model.load(model_path)

    return model, class_labels, input1, conv1, pool1, conv2, conv3, conv4, conv5, conv6, pool2, fc1, drop1, fc2

Copy the unit test that checks the accuracy per class.

The keras version is lightly different, since I did the image pre-processing outside the model.

In [ ]:
def tfl_correct_positives(category, model):
    files = glob.glob(os.path.join(DATABASE_PATH, category, '*.tiff'))

    failed = 0
    for file in files:
        img = imread(file)  # load ROI
        img = scipy.misc.imresize(img, (32, 32), interp="bicubic").astype(np.float32, casting='unsafe')
        prediction = model.predict(np.expand_dims(img, 0))  # run prediction from silcam_classify
        ind = np.argmax(prediction)  # find the highest score
        if not class_labels[ind] == category:
            failed += 1
    success = 100 - (failed / len(files)) * 100
    return success


def keras_correct_positives(category, model):
    files = glob.glob(os.path.join(DATABASE_PATH, category, '*.tiff'))

    failed = 0
    for file in files:
        img = imread(file)  # load ROI
        img = scipy.misc.imresize(img, (32, 32), interp="bicubic").astype(np.float32, casting='unsafe')
        img = (img - 195.17760394934288) / 56.10742134506719
        prediction = model.predict(np.expand_dims(img, 0))
        ind = np.argmax(prediction)  # find the highest score
        if not class_labels[ind] == category:
            failed += 1
    success = 100 - (failed / len(files)) * 100
    return success

In [ ]:
class_labels = get_class_labels(MODEL_PATH)

In [ ]:
model, class_labels, input1, conv1, pool1, conv2, conv3, conv4, conv5, conv6, pool2, fc1, drop1, fc2 = load_model(MODEL_PATH)

Now check the loaded TFL model that it give the accuracies expected.

This take a couple mins to run, and should give:
```
oil 99.40387481371089
other 98.08389435525635
bubble 99.96206373292868
faecal_pellets 96.69260700389106
copepod 99.39117199391173
diatom_chain 98.3529411764706
oily_gas 97.7035490605428
```

In [ ]:
# for c in class_labels:
#     acc = tfl_correct_positives(c, model)
#     print(c, acc)

# Get the weights, transfer to Keras

In [ ]:
# Getting the weights for a specific layer (.W gives weights, .b give biases):
a = model.get_weights(conv1.W)
print(type(a), a.shape)

In [ ]:
all_weights = []
for l in [conv1, conv2, conv3, conv4, conv5, conv6, fc1, fc2]:
    ws = model.get_weights(l.W)
    bs = model.get_weights(l.b)
    all_weights.append([ws, bs])

# Build the Keras model, and transfer the weights

In [ ]:
keras_model = Sequential([
    Conv2D(32, 3, input_shape=(32, 32, 3), activation='relu', padding="same"),
    MaxPooling2D(pool_size=2),
    Conv2D(64, 3, activation='relu', padding="same"),

    # Step 4: Convolution yet again
    Conv2D(64, 3, activation='relu', padding="same"),
    Conv2D(64, 3, activation='relu', padding="same"),
    Conv2D(64, 3, activation='relu', padding="same"),
    Conv2D(64, 3, activation='relu', padding="same"),

    # Step 5: Max pooling again
    MaxPooling2D(pool_size=2),
    
    # Step 6: Fully-connected 512 node neural network
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.25),  # Different from COAP, as the tf states how much to keep, but keras is the dropout rate
    Dense(len(class_labels), activation='softmax'),
])

To save the model, we needed to have values parameters set for loss, so just setting these with the same as the TFL model, although we won't train the model.

In [ ]:
keras_model.compile(keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
keras_model.layers

In [ ]:
# Looking at the weights, we see that they are a len()=2 list with the weights, then bias. Both of those are then np.arrays.
ws = keras_model.layers[0].get_weights()
print(type(ws), type(ws[0]))
print(len(ws))

# For nontrainable layers they are empty lists:
ws = keras_model.layers[1].get_weights()
print(type(ws))
print(ws)

### Actually setting weights:

In [ ]:
# [conv1, conv2, conv3, conv4, conv5, conv6, fc1, fc2]
keras_model.layers[0].set_weights(all_weights[0])
keras_model.layers[2].set_weights(all_weights[1])
keras_model.layers[3].set_weights(all_weights[2])
keras_model.layers[4].set_weights(all_weights[3])
keras_model.layers[5].set_weights(all_weights[4])
keras_model.layers[6].set_weights(all_weights[5])
keras_model.layers[9].set_weights(all_weights[6])
keras_model.layers[11].set_weights(all_weights[7])

In [ ]:
keras_model.save('keras_model.h5')

# Test the 'transferred' model

In [ ]:
# keras_model = keras.models.load_model('keras_model.h5')
loaded_model = keras.models.load_model('keras_model.h5')

In [ ]:
for c in class_labels:
    acc = keras_correct_positives(c, loaded_model)
    print(c, acc)

```
oil 99.40387481371089
other 98.08389435525635
bubble 99.96206373292868
faecal_pellets 96.69260700389106
copepod 99.39117199391173
diatom_chain 98.3529411764706
oily_gas 97.7035490605428 
```

# Some other checks:

In [ ]:
print(model)
print(keras_model)

In [ ]:
# A some what surpurflous check that the weights are the same:
# [conv1, conv2, conv3, conv4, conv5, conv6, fc1, fc2]

assert (model.get_weights(conv1.W) - keras_model.layers[0].get_weights()[0]).sum() == 0, "Oh no, you did an error"
assert (model.get_weights(conv1.b) - keras_model.layers[0].get_weights()[1]).sum() == 0, "Oh no, you did an error"

assert (model.get_weights(conv3.W) - keras_model.layers[3].get_weights()[0]).sum() == 0, "Oh no, you did an error"
assert (model.get_weights(conv3.b) - keras_model.layers[3].get_weights()[1]).sum() == 0, "Oh no, you did an error"

assert (model.get_weights(conv6.W) - keras_model.layers[6].get_weights()[0]).sum() == 0, "Oh no, you did an error"
assert (model.get_weights(conv6.b) - keras_model.layers[6].get_weights()[1]).sum() == 0, "Oh no, you did an error"

assert (model.get_weights(fc1.W) - keras_model.layers[9].get_weights()[0]).sum() == 0, "Oh no, you did an error"
assert (model.get_weights(fc1.b) - keras_model.layers[9].get_weights()[1]).sum() == 0, "Oh no, you did an error"

assert (model.get_weights(fc2.W) - keras_model.layers[11].get_weights()[0]).sum() == 0, "Oh no, you did an error"
assert (model.get_weights(fc2.b) - keras_model.layers[11].get_weights()[1]).sum() == 0, "Oh no, you did an error"

In [ ]:
# Some checked I needed when debugging:
# files = glob.glob(os.path.join(DATABASE_PATH, 'oil', '*.tiff'))
# img = imread(files[4])
# img = scipy.misc.imresize(img, (32, 32), interp="bicubic").astype(np.float32, casting='unsafe')
# img = np.expand_dims(img, 0)

# pp_img = img.copy()
# pp_img = pp_img / (pp_img.std() / 1.186766)
# pp_img = pp_img - (pp_img.mean() + 0.50608945 )
# print(pp_img.mean(), pp_img.std())
# print('Keras prediction:   ', keras_model.predict(pp_img))
# print('TFLearn prediction: ', model.predict(img))
# print('Keras prediction:   ', keras_model.predict(img))
# print(img.mean(), img.std())

# print(keras_model)
# print(model)